In [2]:
:dep caw = { path = "/Users/s/src/caw/caw", features = ["player"] }
:dep caw_midi_udp_widgets_app_lib = { path = "/Users/s/src/caw/midi-udp-widgets-app-lib" }

In [3]:
use caw::prelude::*;
let out = Stereo::new_fn(|| svf32(0.0));
let player: PlayerHandle = play_stereo_default(out.clone()).unwrap();

In [4]:
let tempo_hz = svf32(8.0);
let trig = sv(periodic_trig_hz(tempo_hz.clone()).build());
let seq = {
    use note::*;
    sv(value_sequencer(trig.clone(), [(C2, 0.5f32), (G2, 0.75)]))
};

In [5]:
let lfo = svf32(0.5);
let res = svf32(0.5);
out.set(|| {
    let (note, accent) = seq.clone().unzip();
    let accent = accent.shared();
    let gate = trig.clone().trig_to_gate(0.01).shared();
    let filter_env = adsr_linear_01(gate.clone()).a(0.01).r(0.125).build().exp_01(1.);
    let amp_env = adsr_linear_01(gate.clone()).a(0.01).r(0.125).build().exp_01(1.);
    let osc =
        oscillator(Saw, note.freq_hz() ).build();
    let v =(osc
         
         * amp_env)
    ;
    let d = 
        trig.clone().divide(8).trig(drum::kick()).filter(low_pass::default(6000.0)) * 0.5 +
        trig.clone().divide_with_offset(8, 4).trig(drum::snare())
        .filter(low_pass::default(10000.0)) 
        * 0.5;
    (v
         .filter(compressor().scale(8.).threshold(0.5))
       // .filter(quantizer(0.1))
        .filter(chorus())
        .filter(reverb())
    .filter(delay_s(0.25))
     +d).filter(low_pass::default(filter_env * lfo.clone() * 20_000. ).q(res.clone()))
});

In [8]:
seq.set({
    use note::*;
    value_sequencer(trig.clone(), [
        (G2, 1.0f32),
        (A2, 0.0),
        (A2, 0.0),
        (C2, 0.0),
        (A2, 1.0),
        (B3, 1.0),
        (G2, 1.0),
        (C3, 0.0),
        (D2, 0.0),
        (A3, 0.0),
        (B3, 0.0),
        (C4, 0.2),
    ]) 
});

In [9]:
use caw_midi_udp_widgets_app_lib::*;
res.set(knob("res").controller(0).build() * 4.0);
lfo.set(knob("lfo").controller(1).build());

2025-05-04 12:20:41.984 caw_midi_udp_widgets_app[49654:8794319] +[IMKClient subclass]: chose IMKClient_Modern
2025-05-04 12:20:41.984 caw_midi_udp_widgets_app[49654:8794319] +[IMKInputSession subclass]: chose IMKInputSession_Modern
2025-05-04 12:20:45.519 caw_midi_udp_widgets_app[49627:8794292] +[IMKClient subclass]: chose IMKClient_Modern
2025-05-04 12:20:45.519 caw_midi_udp_widgets_app[49627:8794292] +[IMKInputSession subclass]: chose IMKInputSession_Modern


In [13]:
tempo_hz.set(5.)

()


thread 'main' panicked at src/main.rs:57:22:
called `Result::unwrap()` on an `Err` value: Connection refused (os error 61)

Stack backtrace:
   0: std::backtrace::Backtrace::create
   1: anyhow::error::<impl core::convert::From<E> for anyhow::Error>::from
   2: caw_midi_udp_client::MidiUdpClient::send
   3: caw_midi_udp_widgets_app::main
   4: std::sys::backtrace::__rust_begin_short_backtrace
   5: std::rt::lang_start::{{closure}}
   6: std::rt::lang_start_internal
   7: _main
stack backtrace:
   0: _rust_begin_unwind

thread 'main' panicked at src/main.rs:57:22:
called `Result::unwrap()` on an `Err` value: Connection refused (os error 61)

Stack backtrace:
   0: std::backtrace::Backtrace::create
   1: anyhow::error::<impl core::convert::From<E> for anyhow::Error>::from
   2: caw_midi_udp_client::MidiUdpClient::send
   3: caw_midi_udp_widgets_app::main
   4: std::sys::backtrace::__rust_begin_short_backtrace
   5: std::rt::lang_start::{{closure}}
   6: std::rt::lang_start_internal
   7